In [1]:
import json
import numpy as np
import datetime as dt
import elk as elk
import re
from Credentials import config

In [86]:
# get the logs 
es = elk.ElasticSearch(config.es.host, config.es.cred) # Need to set config first
# es.index(['logstash-victim2*', 'logstash-ad*'])
es.index('logstash-victim2*')

## pth
# es.time('2021-04-17T15:19:19+0800', '2021-04-17T15:22:40+0800') # pth logs
# es.time('2021-05-30T00:00:00+0800', '2021-05-30T00:05:00+0800') # pth logs with eid 3
es.time('2021-05-29T00:00:00+0800', '2021-05-30T00:05:00+0800') # pth logs with eid 3 time long
# es.time('2021-05-26T00:00:00+0800', '2021-05-27T00:00:00+0800') # pth logs with eid 3 time long

## pth with merlin
# es.time('2021-05-15T20:49:00+0800', '2021-05-15T20:54:00+0800') # pth logs with merlin
# es.time('2021-05-15T20:39:00+0800', '2021-05-15T20:54:00+0800') # pth logs with merlin time long

## ptt 
# es.time('2021-03-22T21:08:30+0800', '2021-03-22T21:10:12+0800') # ptt logs

# es.time('2021-05-26T11:34:54+0800', '2021-05-26T11:49:58+0800') # runas /netonly

es.should([{'event.code':'4624'},{'event.code':'10'},{'event.code':'1'},{'event.code':'3'}])
# es.should([{'event.code':'4624'},{'event.code':'10'},{'event.code':'1'},{'event.code':'4768'},\
#           {'event.code':'4769'},{'event.code':'4770'}])
logs = es.search(clean=True)
logs.reverse()
print(len(logs))
# print(data)


D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\elasticsearch\connection\http_urllib3.py:211: UserWarning: Connecting to https://140.113.194.82:9200 using SSL with verify_certs=False is insecure.
  % self.host


7316


In [92]:
def make_lsass_whitelist():
    es = elk.ElasticSearch(config.es.host, config.es.cred) # Need to set config first
    es.index('logstash-victim2*')
#     es.time('now-7d','now')
    es.time('2021-04-26T00:00:00.000Z', '2021-04-19T00:00:00.000Z') # random pick

    es.should([{'event.code':'10'},{'TargetImage':"C:\WINDOWS\system32\lsass.exe"}])
    log_targetImage = es.search(clean=True)

    es = elk.ElasticSearch(config.es.host, config.es.cred) # Need to set config first
    es.index('logstash-victim2*')
    es.time('now-7d','now')
    es.should([{'event.code':'10'},{'SourceImage':"C:\WINDOWS\system32\lsass.exe"}])
    log_sourceImage = es.search(clean=True)
    
    whitelist = set()

    for log in log_targetImage:
        whitelist.add(log['winlog']['event_data']['SourceImage'])

    for log in log_sourceImage:
        whitelist.add(log['winlog']['event_data']['TargetImage'])

    return whitelist

def add_whitelist_0x1010(whitelist):
    whitelist.add("C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2104.14-0\MsMpEng.exe")
    whitelist.add("C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2104.10-0\MsMpEng.exe")
    whitelist.add("C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2103.7-0\MsMpEng.exe")
    
    return whitelist

def remove_whitelist_malicious(whitelist):
    whitelist.remove("C:\\Users\\user\\Desktop\\mimikatz.exe")
    whitelist.remove('C:\\Users\\user\\Downloads\\mimikatz.exe')
    whitelist.remove("C:\\Users\\user\\Desktop\\mimikatz\\mimikatz.exe")
    whitelist.remove('C:\\WINDOWS\\system32\\PsExec.exe')
    
    return whitelist
    
whitelist = make_lsass_whitelist()
whitelist = add_whitelist_0x1010(whitelist)
whitelist = remove_whitelist_malicious(whitelist)
print(whitelist)

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\elasticsearch\connection\http_urllib3.py:211: UserWarning: Connecting to https://140.113.194.82:9200 using SSL with verify_certs=False is insecure.
  % self.host
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\elasticsearch\connection\http_urllib3.py:211: UserWarning: Connecting to https://140.113.194.82:9200 using SSL with verify_certs=False is insecure.
  % self.host


{'C:\\Program Files\\WindowsApps\\Microsoft.YourPhone_1.21042.137.0_x64__8wekyb3d8bbwe\\YourPhone.exe', 'C:\\WINDOWS\\System32\\WaaSMedicAgent.exe', 'C:\\Program Files\\WindowsApps\\Microsoft.MicrosoftOfficeHub_18.2104.12721.0_x64__8wekyb3d8bbwe\\LocalBridge.exe', 'C:\\WINDOWS\\system32\\SppExtComObj.exe', 'C:\\WINDOWS\\system32\\cleanmgr.exe', 'C:\\WINDOWS\\system32\\nltest.exe', 'C:\\WINDOWS\\system32\\SearchProtocolHost.exe', 'C:\\WINDOWS\\system32\\lsass.exe', 'C:\\ProgramData\\Microsoft\\Windows Defender\\Platform\x04.18.2104.14-0\\MsMpEng.exe', 'C:\\Users\\user\\AppData\\Local\\Microsoft\\OneDrive\\OneDriveStandaloneUpdater.exe', 'C:\\Windows\\Microsoft.NET\\Framework\\v4.0.30319\\ngen.exe', 'C:\\WINDOWS\\system32\\mmc.exe', 'C:\\WINDOWS\\system32\\taskhostw.exe', 'C:\\Program Files (x86)\\Internet Explorer\\IEXPLORE.EXE', 'C:\\WINDOWS\\System32\\svchost.exe', 'C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe', 'C:\\ProgramData\\Microsoft\\Windows Defender\\Platform\\4.1

In [87]:
## remove whitelist log from logs

"""
concern: 
some logs in the attack which will also be remove, but they don't impact on the detection. 
e.g. 
10 C:\WINDOWS\system32\lsass.exe → C:\WINDOWS\system32\svchost.exe 0x1000
10 C:\WINDOWS\system32\svchost.exe → C:\WINDOWS\system32\lsass.exe 0x1000
     10 C:\WINDOWS\system32\lsass.exe → C:\WINDOWS\system32\svchost.exe 0x1000
"""

for i in range(5):
    for log in logs:
        if log['event']['code'] == 10:
            if log['winlog']['event_data']['SourceImage'] == "C:\WINDOWS\system32\lsass.exe":
                if log['winlog']['event_data']['TargetImage'] in whitelist:
                    logs.remove(log)
            elif log['winlog']['event_data']['TargetImage'] == "C:\WINDOWS\system32\lsass.exe":
                if log['winlog']['event_data']['SourceImage'] in whitelist:
                    logs.remove(log)
                
print(len(logs))

4474


In [63]:
class struct:
    #__slots__ = []
    def __init__(self,**data):
        self.__dict__.update(data)
        self.next = None
        
class treeNode:
    def __init__(self, SourceProcessGUID):
        self.root = True
        self.EID = None
        self.SourceProcessGUID = SourceProcessGUID
        self.ChildProcessGUID = None
        self.SourceImage = None
        self.ChildImage = None
        self.GrantedAccess = None
        self.LogonId = None
        self.SubjectLogonId = None
        self.TargetLogonId = None
        self.LogonType = None
        self.time = None
        self.Parent = None
        
        self.Children = []
        
        ## network
        self.Protocol = None
        self.DestinationIp = None
        self.SourcePort = None
        self.DestinationPort = None
        self.SourceIp = None

In [88]:
print("len of logs:", len(logs))

def group_with_guid():
    first = True
    trees = []

    for log in logs:
        c = False
        trees.reverse()
        for t in trees:
            if log['event']['code'] == 10:
                if log['winlog']['event_data']['SourceProcessGUID'] == t.ChildProcessGUID \
                    or log['winlog']['event_data']['TargetProcessGUID'] == t.ChildProcessGUID:
                    tree = treeNode(log['winlog']['event_data']['SourceProcessGUID'])
                    tree.EID = 10
                    tree.ChildProcessGUID = log['winlog']['event_data']['TargetProcessGUID']
                    tree.SourceImage = log['winlog']['event_data']['SourceImage']
                    tree.ChildImage = log['winlog']['event_data']['TargetImage']
                    tree.GrantedAccess = log['winlog']['event_data']['GrantedAccess']
                    tree.time = log['event']['created']
                    tree.root = False
                    tree.Parent = t
                    t.Children.append(tree)
                    trees.reverse()
                    trees.append(tree)
                    c = True
                    break
            elif log['event']['code'] == 1 and log['winlog']['event_data']['ParentProcessGuid'] == t.ChildProcessGUID:
                tree = treeNode(log['winlog']['event_data']['ParentProcessGuid'])
                tree.EID = 1
                tree.ChildProcessGUID = log['winlog']['event_data']['ProcessGuid']
                tree.SourceImage = log['winlog']['event_data']['ParentImage']
                tree.ChildImage = log['winlog']['event_data']['Image']
                tree.LogonId = log['winlog']['event_data']['LogonId']
                tree.time = log['event']['created']
                tree.root = False
                t.Children.append(tree)
                trees.reverse()
                trees.append(tree)
                c = True
                break
                
            ## for the logon type 9
            elif log['event']['code'] == 4624 and log['winlog']['event_data']['SubjectLogonId'] == t.LogonId:
                tree = treeNode('{4624}')
                tree.root = False
                tree.EID = 4624
                tree.SubjectLogonId = log['winlog']['event_data']['SubjectLogonId']
                tree.TargetLogonId = log['winlog']['event_data']['TargetLogonId']
                tree.LogonType = log['winlog']['event_data']['LogonType']
                tree.time = log['event']['created']
                t.Children.append(tree)
                trees.reverse()
                trees.append(tree)
                c = True
                break
                
            ## for the network destination detection
            elif log['event']['code'] == 3 and log['winlog']['event_data']['ProcessGuid'] == t.ChildProcessGUID:
                tree = treeNode(log['winlog']['event_data']['ProcessGuid'])
                tree.EID = 3
                tree.Image = log['winlog']['event_data']['Image']
                tree.Protocol = log['winlog']['event_data']['Protocol']
                tree.DestinationIp = log['winlog']['event_data']['DestinationIp']
                tree.SourcePort = log['winlog']['event_data']['SourcePort']
                tree.DestinationPort = log['winlog']['event_data']['DestinationPort']
                tree.SourceIp = log['winlog']['event_data']['SourceIp']
                tree.root = False
                t.Children.append(tree)
                trees.reverse()
                trees.append(tree)
                c = True
                break

        if c:
            continue
        trees.reverse()

        if log['event']['code'] == 1:
            tree = treeNode(log['winlog']['event_data']['ParentProcessGuid'])
            tree.EID = 1
            tree.ChildProcessGUID = log['winlog']['event_data']['ProcessGuid']
            tree.SourceImage = log['winlog']['event_data']['ParentImage']
            tree.ChildImage = log['winlog']['event_data']['Image']
            tree.LogonId = log['winlog']['event_data']['LogonId']
            tree.time = log['event']['created']
            trees.append(tree)
    
    return trees

trees = group_with_guid()
print(trees)

len of logs: 4474
[<__main__.treeNode object at 0x000001AD152A9438>, <__main__.treeNode object at 0x000001AD152A9080>, <__main__.treeNode object at 0x000001AD143F2668>, <__main__.treeNode object at 0x000001AD143F26D8>, <__main__.treeNode object at 0x000001AD14496E10>, <__main__.treeNode object at 0x000001AD143F2710>, <__main__.treeNode object at 0x000001AD14496E80>, <__main__.treeNode object at 0x000001AD14496EB8>, <__main__.treeNode object at 0x000001AD14496978>, <__main__.treeNode object at 0x000001AD144969E8>, <__main__.treeNode object at 0x000001AD14490320>, <__main__.treeNode object at 0x000001AD14496A20>, <__main__.treeNode object at 0x000001AD14490390>, <__main__.treeNode object at 0x000001AD144F72B0>, <__main__.treeNode object at 0x000001AD144F7320>, <__main__.treeNode object at 0x000001AD144F7358>, <__main__.treeNode object at 0x000001AD144903C8>, <__main__.treeNode object at 0x000001AD14049470>, <__main__.treeNode object at 0x000001AD14049518>, <__main__.treeNode object at 0x

In [10]:
def ptt_detect():
    ptt_detect = None
    current = None
    First = True
    
    for log in logs:
        if First:
            if log['event']['code'] == 4768 or log['event']['code'] == 4769 or log['event']['code'] == 4770:
                node = struct()
                node.eid = log['event']['code']
                node.time = log['event']['created']
                node.next = None
                current = node
                ptt_detect = current
                First = False
        else:
            if log['event']['code'] == 4768 or log['event']['code'] == 4769 or log['event']['code'] == 4770:
                node = struct()
                node.eid = log['event']['code']
                node.time = log['event']['created']
                node.next = None
                current.next = node
                current = current.next

    return ptt_detect

ptt_trees = ptt_detect()
print(ptt_trees)

None


In [11]:
def print_ptt_trees(trees):
    
    current = trees
    First = True
    while current != None:
        if First:
            print(current.eid, current.time, end="")
            First = False
        else:
            print(" →", current.eid, current.time, end="")
        current = current.next

print_ptt_trees(ptt_trees)

In [89]:
def print_tree(tress):
        
    def print_child(tree, i):
        for t in tree:
                if t.EID == 4624:
                    print('    '*i, t.EID, end=" ")
                    print("LogonType:", t.LogonType)
                    continue
    #             print('\t'*i, t.EID, t.time, t.SourceImage,'→', t.ChildImage, end=" ")
                elif t.EID == 3:
                    print('    '*i, t.EID, end=" ")
                    print("Destination IP:", t.DestinationIp, "Destination Port:", t.DestinationPort)
                    continue
                print('    '*i, t.EID, t.SourceImage,'→', t.ChildImage, end=" ")
    #             print('\t'*i, t.EID, t.SourceImage, t.SourceProcessGUID, '→', t.ChildImage, t.ChildProcessGUID, end=" ")
                if t.EID == 10:
                    print(t.GrantedAccess)
                else:
                    print()
                print_child(t.Children, i+1)

    for tree in trees:
        if tree.root:
            print(tree.EID, tree.time, tree.SourceImage, '→', tree.ChildImage, end=" ")
#             print(tree.EID, tree.SourceImage, '→', tree.ChildImage, end=" ")
    #         print(tree.EID, tree.SourceImage, tree.SourceProcessGUID, '→', tree.ChildImage, tree.ChildProcessGUID, end=" ")
            if tree.EID == 10:
                print(tree.GrantedAccess)
            else:
                print()
            print_child(tree.Children, 1)
            print()
            
print_tree(trees)

1 2021-05-25T16:02:18.807Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-25T16:02:18.807Z C:\Windows\System32\services.exe → C:\Windows\servicing\TrustedInstaller.exe 

1 2021-05-25T16:02:18.807Z C:\Windows\System32\svchost.exe → C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.19041.985_none_e72c6fe7263b0fe4\TiWorker.exe 
     4624 LogonType: 5

1 2021-05-25T16:06:21.659Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-25T16:06:21.659Z C:\Windows\System32\services.exe → C:\Windows\servicing\TrustedInstaller.exe 

1 2021-05-25T16:06:21.659Z C:\Windows\System32\svchost.exe → C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.19041.985_none_e72c6fe7263b0fe4\TiWorker.exe 

1 2021-05-25T16:11:22.769Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-25T16:13:00.150Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 
    

                 3 Destination IP: 192.168.1.113 Destination Port: 88
                 3 Destination IP: 192.168.1.113 Destination Port: 88
                 3 Destination IP: 192.168.1.113 Destination Port: 135
                 3 Destination IP: 192.168.1.113 Destination Port: 49668
                 3 Destination IP: 192.168.1.113 Destination Port: 49671
                 3 Destination IP: 192.168.1.113 Destination Port: 389
                 3 Destination IP: 192.168.1.113 Destination Port: 49668
                 3 Destination IP: 192.168.1.113 Destination Port: 135
                 10 C:\WINDOWS\system32\svchost.exe → C:\WINDOWS\system32\lsass.exe 0x1000
                     3 Destination IP: 192.168.1.113 Destination Port: 135
                     3 Destination IP: 192.168.1.113 Destination Port: 389
                     3 Destination IP: 192.168.1.113 Destination Port: 49668
                     3 Destination IP: 192.168.1.113 Destination Port: 389
                     3 Destination 

1 2021-05-26T03:47:35.515Z C:\Windows\System32\svchost.exe → C:\Windows\System32\smartscreen.exe 
     3 Destination IP: 13.67.52.249 Destination Port: 443

1 2021-05-26T03:47:35.515Z C:\Windows\explorer.exe → C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe 
     1 C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe → C:\Windows\System32\conhost.exe 
     1 C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe → C:\Windows\System32\runas.exe 
         4624 LogonType: 9
         1 C:\Windows\System32\runas.exe → C:\Windows\System32\cmd.exe 
             1 C:\Windows\System32\cmd.exe → C:\Windows\System32\conhost.exe 
             1 C:\Windows\System32\cmd.exe → C:\Windows\System32\whoami.exe 
             1 C:\Windows\System32\cmd.exe → C:\Windows\System32\PsExec.exe 
                 10 C:\WINDOWS\system32\lsass.exe → C:\WINDOWS\system32\PsExec.exe 0x1000
                     10 C:\WINDOWS\system32\lsass.exe → C:\WINDOWS\system32\PsExec.exe 0x1478
                  

         1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngentask.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe 
             1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\mscorsvw.exe 
         1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngentask.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe 
             1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\mscorsvw.exe 
         1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngentask.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe 
             1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\mscorsvw.exe 
         1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngentask.exe → C:\Windows\Microsoft.NET\Framework64\v4.0.30319\ngen.exe 
             1 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\n


1 2021-05-26T08:16:50.858Z C:\Windows\System32\svchost.exe → C:\Windows\System32\backgroundTaskHost.exe 

1 2021-05-26T08:17:23.995Z C:\Windows\System32\svchost.exe → C:\Windows\System32\backgroundTaskHost.exe 

1 2021-05-26T08:17:23.995Z C:\Windows\System32\svchost.exe → C:\Windows\System32\RuntimeBroker.exe 

1 2021-05-26T08:17:51.091Z C:\Windows\System32\svchost.exe → C:\Windows\System32\backgroundTaskHost.exe 

1 2021-05-26T08:19:23.456Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-26T08:19:23.457Z C:\Windows\System32\services.exe → C:\Windows\servicing\TrustedInstaller.exe 

1 2021-05-26T08:19:23.457Z C:\Windows\System32\svchost.exe → C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.19041.985_none_e72c6fe7263b0fe4\TiWorker.exe 
     4624 LogonType: 5

1 2021-05-26T08:30:25.767Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-26T08:30:25.768Z C:\Windows\System32\services.exe → C:\Windo

     3 Destination IP: 204.79.197.200 Destination Port: 443

1 2021-05-26T12:18:27.355Z C:\Windows\System32\svchost.exe → C:\Windows\System32\backgroundTaskHost.exe 

1 2021-05-26T12:30:00.720Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-26T12:30:00.721Z C:\Windows\System32\services.exe → C:\Windows\servicing\TrustedInstaller.exe 

1 2021-05-26T12:30:00.721Z C:\Windows\System32\svchost.exe → C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31bf3856ad364e35_10.0.19041.985_none_e72c6fe7263b0fe4\TiWorker.exe 
     4624 LogonType: 5

1 2021-05-26T12:32:53.294Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-26T12:33:27.430Z C:\Windows\System32\svchost.exe → C:\Windows\System32\taskhostw.exe 

1 2021-05-26T12:33:27.430Z C:\Windows\System32\services.exe → C:\Windows\servicing\TrustedInstaller.exe 

1 2021-05-26T12:33:27.430Z C:\Windows\System32\svchost.exe → C:\Windows\WinSxS\amd64_microsoft-windows-servicingstack_31